In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import json

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [3]:
#Write out URLs
url_mars_news='https://mars.nasa.gov/news/'
url_space_images='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url_facts='https://space-facts.com/mars/'
url_hemi='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [12]:
#Parse Mars News
browser.visit(url_mars_news)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
article=soup.find_all('ul', class_='item_list')[0].find_all('li',class_='slide')

title=[]
news=[]
for i in range(len(article)):
    title.append(article[i].find('div',class_='content_title').text)
    news.append(article[i].find('div',class_='article_teaser_body').text)

In [64]:
#Parse Space Images
browser.visit(url_space_images)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
article=soup.find_all('ul', class_='articles')[0].find_all('li',class_='slide')[0]
featured_image_url='https://www.jpl.nasa.gov'+article.find('div', class_='img').img['src']

In [96]:
#Parse facts
browser.visit(url_facts)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

article=soup.table.find_all('tr')

table_data=pd.DataFrame({'Property':[],'Value':[]})
for i in range(len(article)):
    p=article[i].find('td', class_='column-1').text
    v=article[i].find('td', class_='column-2').text
    table_data=table_data.append(pd.DataFrame({'Property':[p],'Value':[v]}))

table_data=table_data.reset_index(drop=True)
table_data

,Property,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [127]:
#Covert to HTML String

html_data=table_data.to_html()
text_file = open("data_table.html", "w")
text_file.write(html_data)
text_file.close()

In [118]:
#Parse Hemispheres

hemsisphere_url=pd.DataFrame({'Title':[], 'Img_url':[]})

for i in range(4):
    browser.visit(url_hemi)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    article=soup.find_all('div', class_='item')
    link=article[i].a['href']
    browser.visit('https://astrogeology.usgs.gov'+link)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img_url=soup.li.a['href']
    img_name=str.split(soup.h2.text)
    hemsisphere_url=hemsisphere_url.append(pd.DataFrame({'Title':[img_name[0]+' '+img_name[1]], 'Img_url':[img_url]}))
hemsisphere_url=hemsisphere_url.reset_index(drop=True)
hemsisphere_url

,Title,Img_url
0,Cerberus Hemisphere,https://astropedia.astrogeology.usgs.gov/downl...
1,Schiaparelli Hemisphere,https://astropedia.astrogeology.usgs.gov/downl...
2,Syrtis Major,https://astropedia.astrogeology.usgs.gov/downl...
3,Valles Marineris,https://astropedia.astrogeology.usgs.gov/downl...


In [114]:
img_name=str.split(soup.h2.text)
#hemsisphere_url=pd.DataFrame({'Title':[], 'Img_url':[]})


['Cerberus', 'Hemisphere', 'Enhanced']

In [126]:
hemsisphere_url['Img_url'][3]

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'